

- How to deal with different columns in generation data for old (`DE_AT_LU` until 2018/09/30) and new bidding zone (`DE_LU` since 2018/10/01)? Old data contains all columns from new data but also additional columns, mostly about `'Actual Consumption'`, and one extra category `'Fossil Coal-derived gas Actual Aggregated'`.
- Which time span to include in general for training data?

## Data-loading playground with `entsoe-py`

### Imports

In [1]:
import pandas as pd
import plotly.express as px
pd.set_option('display.max_rows', 100)

### Settings

In [2]:
api_key = "6e68642c-8403-4caa-af31-bda40b8c67f6" # web token for RESTful API
country_code = "10Y1001A1001A83F" # Germany
BZ_code = "DE_LU" # new bidding zone, valid since 2018/10/01
BZ_code_old = "DE_AT_LU" # old bidding zone, valid until 2018/09/30
time_zone = "Europe/Berlin" # time zone for Germany

### Helper functions

In [3]:
def get_col_diff_intersect(df1, df2):
    """
    Return difference and intersection of columns of two dataframes.
    
    Params
    ------
    df1 : pandas.DataFrame
          first dataframe
    df2 : pandas.DataFrame
          second dataframe
          
    Returns
    -------
    difference in columns of df1 and df2
    intersection of columns of df1 and df2
    
    """
    return df1.columns.difference(df2.columns), df1.columns.intersection(df2.columns)

In [4]:
def get_load_intervals(start_date, 
                       end_date, 
                       time_zone="Europe/Berlin"):
    """
    Get time points for sequential data loading from ENTSO-E transparency platform.
    
    For one request, the time delta for loading data from the platform is limited to one year.
    
    Params
    ------
    start_date : str
                 start date as "yyyymmdd"
    end_date : str
               end date as "yyyymmdd"
    time_zone : str
                time zone as string, e.g. "Europe/Berlin"
    
    Returns
    -------
    pd.Series
    pandas series with timestamps of time points to consider between start and end date
    """
    # Convert start and end dates to timestamps.
    start = pd.Timestamp(start_date, tz=time_zone)
    end = pd.Timestamp(end_date, tz=time_zone)

    # Create series from start and end timestamps.
    start_series = pd.Series(pd.Timestamp(start_date))
    end_series = pd.Series(pd.Timestamp(end_date))
    
    # Create date range from start and end dates and determine year starts within range.
    # Convert data range to series.
    dates = pd.date_range(start=start_date, end=end_date, freq="YS", inclusive="both").to_series()

    # Check whether start date itself is year start.
    # If not, prepend to dates to consider for data loading.
    if not start.is_year_start:
        dates = pd.concat([start_series, dates], ignore_index=True)

    # Check whether end date itself is year start.
    # If not, append to dates to consider for data loading.
    if not end.is_year_start:
        dates = pd.concat([dates, end_series], ignore_index=True)
        
    return dates

In [5]:
def load_data(start_date, 
              end_date, 
              api_key, 
              country_code="10Y1001A1001A83F", 
              time_zone="Europe/Berlin"):
    """
    Load actual load and actual aggregated generation per production type for requested time interval.
    
    
    Params
    ------
    start_date : str
                 start date as "yyyymmdd"
    end_date : str
               end date as "yyyymmdd"
    api_key : str
              RESTful API web key
    country_code : str
                   code for country, bidding zone, etc.
    time_zone : str
                time zone as string, e.g. "Europe/Berlin"
                
    Returns
    -------
    pd.DataFrame with time points as indices and load + generation per type as columns.
    """
    from entsoe import EntsoePandasClient
    # Initialize client and settings.
    client = EntsoePandasClient(api_key=api_key)
    start = pd.Timestamp(start_date, tz=time_zone)
    end = pd.Timestamp(end_date, tz=time_zone)
    # Query data and save to dataframe.
    df_load = client.query_load(country_code, start=start, end=end)
    print(f"Actual load has shape {df_load.shape}.")
    df_gen = client.query_generation(country_code, start=start, end=end, psr_type=None)
    df_gen.columns = [" ".join(a) for a in df_gen.columns.to_flat_index()]
    print(f"Actual generation per production type has shape {df_gen.shape}.")
    df_final = pd.concat([df_load, df_gen], axis=1) # Concatenate dataframes in columns dimension.
    print(f"Concatenated data frame has shape {df_final.shape}.")
    
    return df_final

In [6]:
def fetch_data(start_date, 
               end_date, 
               api_key, 
               country_code="10Y1001A1001A83F", 
               time_zone="Europe/Berlin",
               drop_consumption=True,
               create_PSLP_columns=True):
    """
    Fetch data from ENTSO-E transparency platform as requested.
    
    Parameters
    ----------
    start_date : str
                 start date as "yyyymmdd"
    end_date : str
               end date as "yyyymmdd"
    api_key : str
              RESTful API web key
    time_zone : str
                time zone as string, e.g. "Europe/Berlin"
    country_code : str
                   code for country, bidding zone, etc.
    drop_consumption : Bool
                       Drop columns containing actual consumption.
    
    Returns
    -------
    pd.DataFrame with actual load and generation per type for requested time interval
    """
    # Determine sequence of dates to consider when loading data.
    dates = get_load_intervals(start_date, end_date, time_zone)
    print(f"Consider the following dates:\n{dates}")
    df_list = []
    
    for i, _ in enumerate(dates):

        if i == dates.shape[0] - 1:
            print("Returning final data frame...")
            df_final = pd.concat(df_list, axis=0) # Concatenate dataframes along time axis (index).
            df_final.index = pd.to_datetime(df_final.index, utc=True).tz_convert(tz="UTC+01:00")

            # Drop columns containing actual consumption?
            if drop_consumption:
                print("Dropping columns containing actual consumption...")
                df_final.drop(list(df_final.filter(regex='Consumption')), axis=1, inplace=True)
            original_headers = df_final.columns
            # Create PSLP columns?
            if create_PSLP_columns:
                print("Creating columns for PSLP calculation...")
                for header in original_headers:
                    df_final[str(header) + " PSLP"] = pd.Series(dtype='float')
            return df_final, original_headers
            
        try:
            print(f"Trying to load data chunk for time interval [{dates[i]}, {dates[i+1]}]...")
            df_temp = load_data(start_date=dates[i], 
                                end_date=dates[i+1],
                                api_key=api_key,
                                time_zone=time_zone,
                                country_code=country_code)
            print(df_temp.shape)
            df_list.append(df_temp)
            print("Loading successful!")
            
        except Exception as e:
            print(f"Loading failed!", e)
            continue

In [7]:
start_date = "20171225"
end_date = "20181225"
df_test, original_headers = fetch_data(start_date, end_date, api_key)

Consider the following dates:
0   2017-12-25
1   2018-01-01
2   2018-12-25
dtype: datetime64[ns]
Trying to load data chunk for time interval [2017-12-25 00:00:00, 2018-01-01 00:00:00]...
Actual load has shape (672, 1).
Actual generation per production type has shape (672, 32).
Concatenated data frame has shape (672, 33).
(672, 33)
Loading successful!
Trying to load data chunk for time interval [2018-01-01 00:00:00, 2018-12-25 00:00:00]...
Actual load has shape (34368, 1).
Actual generation per production type has shape (34368, 25).
Concatenated data frame has shape (34368, 26).
(34368, 26)
Loading successful!
Returning final data frame...
Dropping columns containing actual consumption...
Creating columns for PSLP calculation...


In [8]:
def _correct_time_shift(df, usual_length=96):
    unique_dates = df.index.to_series().dt.date.drop_duplicates().tolist() # Get unique dates in data index.
    unique_dates = [pd.to_datetime(d).strftime('%Y-%m-%d') for d in unique_dates]
    time_shift_dates = []
    for date in unique_dates:
        length = df.loc[date].shape[0]
        if length != usual_length:
            print(f"Time shift at {date}, length is {length}.")
            time_shift_dates.append([date, length])
    return time_shift_dates

In [9]:
print(df_test.shape)
print(df_test.columns)
print(df_test["Actual Load"].isna().sum())
print(original_headers)
time_shifted = _correct_time_shift(df_test)

(35040, 36)
Index(['Actual Load', 'Biomass Actual Aggregated',
       'Fossil Brown coal/Lignite Actual Aggregated',
       'Fossil Coal-derived gas Actual Aggregated',
       'Fossil Gas Actual Aggregated', 'Fossil Hard coal Actual Aggregated',
       'Fossil Oil Actual Aggregated', 'Geothermal Actual Aggregated',
       'Hydro Pumped Storage Actual Aggregated',
       'Hydro Run-of-river and poundage Actual Aggregated',
       'Hydro Water Reservoir Actual Aggregated', 'Nuclear Actual Aggregated',
       'Other Actual Aggregated', 'Other renewable Actual Aggregated',
       'Solar Actual Aggregated', 'Waste Actual Aggregated',
       'Wind Offshore Actual Aggregated', 'Wind Onshore Actual Aggregated',
       'Actual Load PSLP', 'Biomass Actual Aggregated PSLP',
       'Fossil Brown coal/Lignite Actual Aggregated PSLP',
       'Fossil Coal-derived gas Actual Aggregated PSLP',
       'Fossil Gas Actual Aggregated PSLP',
       'Fossil Hard coal Actual Aggregated PSLP',
       'Fossil O

In [10]:
#df_test.index = pd.to_datetime(df_test.index, utc=True).tz_convert(tz="UTC+01:00")
#print(df_test.iloc[0])
#print(df_test.index[0])
#print(df_test.loc["2018-03-25"].index[0], df_test.loc["2018-03-25"].index[0].tz, 
#      df_test.loc["2018-03-25"].index[0].utcoffset())
#print(df_test.loc["2018-03-25"].index[8], df_test.loc["2018-03-25"].index[8].tz, 
#      df_test.loc["2018-03-25"].index[8].utcoffset())

In [20]:
#time_shifted_corr = _correct_time_shift(df_test)
print(time_shifted)

[]


In [12]:
def get_pslp_category(date, weekday=None, holiday=None, country_code='DE'):
    """
    Get PSLP category from date, weekday information, and holiday information.
    0 : weekday
    1 : Saturday
    2 : Sunday and holiday
    
    Params
    ------
    date : str
           date in 'YYYYMMDD' format
    weekday : int
              corresponding weekday
              0 - Mon, 1 - Tue, 2 - Wed, 3 - Thu, 4 - Fri, 5 - Sat, 6 - Sun
    holiday : Bool
              True if public holiday, False if not.
    
    Returns
    -------
    int : PSLP category
    """
    # Convert string-type date to datetime object.
    if type(date) is str:
        date = pd.to_datetime(date)
    
    # Assign weekday if not given.
    if weekday is None:
        weekday = date.weekday()
    
    # Assign holiday category if not given.
    if holiday is None:
        import holidays
        holiday = date in holidays.country_holidays(country_code)
    
    # Special treatment for Christmas eve and New year's eve as Saturdays.
    if ( date.day == 24 or date.day == 31 ) and date.month == 12 and weekday != 6:
        pslp_category = 1
    # weekdays
    elif weekday < 5 and holiday is False:
        pslp_category = 0
    # Saturdays
    elif weekday == 5 and holiday is False:
        pslp_category = 1
    # Sundays and holidays
    elif weekday == 6 or holiday is True:
        pslp_category = 2
    return pslp_category

In [13]:
def assign_pslp_categories(df, country_code='DE'):
    """
    Assign PSLP categories to dates in dataframe's datetime index.

    0 is weekday, 1 is Saturday, 2 is Sunday or holiday.
    Special treatment for Christmas eve and New Year's eve (as Saturdays).

    Params
    ------
    df : pandas.Dataframe
    country_code : str
                   country to determine holidays for
    Returns
    -------
    pandas.Dataframe
    Dataframe amended by weekday information, holiday information, and PSLP category
    """

    import holidays
    
    # Get holidays in specified country.
    country_holidays = holidays.country_holidays(country_code) # Passing a state is also possible!

    s = df.index.to_series()                                # Convert datetime index to series.
    dates = s.dt.date                                       # Get plain dates from datetime objects.
    weekdays = s.dt.weekday                                 # Get weekdays from datetime objects.
    holidays = [date in country_holidays for date in dates] # Determine holidays.
    pslp_category = []
    
    for d, wd, hd in zip(dates, weekdays, holidays):
        pslp_category.append(get_pslp_category(d, wd, hd))
        
    df["PSLP Category"] = pslp_category
    df["Holiday"] = holidays
    df["Weekday"] = weekdays
    return df

In [14]:
date_str = "20181028"
df_test = assign_pslp_categories(df_test)
print(df_test.loc[date_str])
print(df_test.columns)
print(df_test["PSLP Category"])

                           Actual Load  Biomass Actual Aggregated  \
2018-10-28 00:00:00+01:00      43205.0                     4720.0   
2018-10-28 00:15:00+01:00      42955.0                     4712.0   
2018-10-28 00:30:00+01:00      42393.0                     4719.0   
2018-10-28 00:45:00+01:00      42369.0                     4723.0   
2018-10-28 01:00:00+01:00      41684.0                     4722.0   
2018-10-28 01:15:00+01:00      41476.0                     4727.0   
2018-10-28 01:30:00+01:00      41310.0                     4725.0   
2018-10-28 01:45:00+01:00      41179.0                     4716.0   
2018-10-28 02:00:00+01:00      41046.0                     4728.0   
2018-10-28 02:15:00+01:00      40891.0                     4725.0   
2018-10-28 02:30:00+01:00      41013.0                     4726.0   
2018-10-28 02:45:00+01:00      40933.0                     4717.0   
2018-10-28 03:00:00+01:00      41167.0                     4725.0   
2018-10-28 03:15:00+01:00      412

In [15]:
def _calculate_pslp_old(df, original_headers, date_str, lookback=3, country_code='DE', DEBUG=False):
    """
    Calculate PSLPs for all dates in dataframe or for given date from given data.
    
    The data is categorized into weekdays, Saturdays, and Sundays/holidays.
    The `lookback` most recent days from the specified date's category are used to
    calculate the corresponding PSLP as the average.
    
    Params
    ------
    df : pandas.Dataframe
         data to calculate PSLP for, must have datetime index
    original_headers : list of str
                       categories to calculate PSLP for
    date_str : str
               date 'YYYYMMDD' to calculate PSLP for; if None, calculate PSLP for all dates
    lookback : int
               number of days to consider in each category for calculating PSLP
    country_code : str
                   considered country (for holidays)
    
    """
    unique_dates = df.index.to_series().dt.date.drop_duplicates().tolist() # Get unique dates in data index.
    df = assign_pslp_categories(df, country_code)

    print(f"Calculating PSLP for date {date_str}...")
    date = pd.to_datetime(date_str)

    # Check whether date is in range of given dataframe.
    if date.date() < unique_dates[0]:
        raise IndexError(f"PSLP cannot be calculated. Date {date_str} is in the past of data.")
    if date.date()  > unique_dates[-1] + pd.Timedelta(days = 1):
        raise IndexError(f"PSLP cannot be calculated. Date {date_str} is more than 1d in the future of data.")
    assert date.date() in unique_dates

    pslp_category = get_pslp_category(date_str)
    print(f"PSLP category of {date.date()} is {pslp_category}.")
    df_pslp = df[df['PSLP Category'] == pslp_category]
    
    unique_dates_pslp = df_pslp.index.to_series().dt.date.drop_duplicates().tolist()
    idx_pslp = unique_dates_pslp.index(date.date())
    print(f"Index in unique days of PSLP category is {idx_pslp}.")
    if idx_pslp - lookback < 0:
        raise IndexError(f"PSLP cannot be calculated. Less than {lookback} samples in PSLP category for date {date_str}.")
    lookback_dates = unique_dates_pslp[idx_pslp-lookback:idx_pslp]
    lookback_dates = [pd.to_datetime(d).strftime('%Y-%m-%d') for d in lookback_dates]
    print(f"Dates to consider for calculating PSLP: {lookback_dates}")
    for header in original_headers:
        print(f"Looking at {header}...")
        for idx, d in enumerate(lookback_dates):
            if idx == 0:
                df[header + " PSLP"].at[date_str] = df[header].loc[d]
                if DEBUG:
                    print(df[header + " PSLP"].at[date_str])
            else:
                temp1 = df[header + " PSLP"].at[date_str].reset_index(drop=True)
                temp2 = df[header].at[d].reset_index(drop=True)
                temp = temp1 + temp2
                df[header + " PSLP"].at[date_str] = temp
                if DEBUG:
                    print(df[header + " PSLP"].at[date_str])
        df[header + " PSLP"].at[date_str] /= lookback
    return df

In [26]:
def _calculate_pslp(df, original_headers, date_str, lookback=3, country_code='DE', DEBUG=False):
    """
    Calculate PSLPs for all dates in dataframe or for given date from given data.
    
    The data is categorized into weekdays, Saturdays, and Sundays/holidays.
    The `lookback` most recent days from the specified date's category are used to
    calculate the corresponding PSLP as the average.
    
    Params
    ------
    df : pandas.Dataframe
         data to calculate PSLP for, must have datetime index
    original_headers : list of str
                       categories to calculate PSLP for
    date_str : str
               date 'YYYYMMDD' to calculate PSLP for; if None, calculate PSLP for all dates
    lookback : int
               number of days to consider in each category for calculating PSLP
    country_code : str
                   considered country (for holidays)
    
    """
    unique_dates = df.index.to_series().dt.date.drop_duplicates().tolist() # Get unique dates in data index.
    df = assign_pslp_categories(df, country_code)

    print(f"Calculating PSLP for date {date_str}...")
    date = pd.to_datetime(date_str)

    # Check whether date is in range of given dataframe.
    if date.date() < unique_dates[0]:
        raise IndexError(f"PSLP cannot be calculated. Date {date_str} is in the past of data.")
    if date.date()  > unique_dates[-1] + pd.Timedelta(days = 1):
        raise IndexError(f"PSLP cannot be calculated. Date {date_str} is more than 1d in the future of data.")
    assert date.date() in unique_dates

    pslp_category = get_pslp_category(date_str)
    print(f"PSLP category of {date.date()} is {pslp_category}.")
    df_pslp = df[df['PSLP Category'] == pslp_category]
    
    unique_dates_pslp = df_pslp.index.to_series().dt.date.drop_duplicates().tolist()
    idx_pslp = unique_dates_pslp.index(date.date())
    print(f"Index in unique days of PSLP category is {idx_pslp}.")
    if idx_pslp - lookback < 0:
        raise IndexError(f"PSLP cannot be calculated. Less than {lookback} samples in PSLP category for date {date_str}.")
    lookback_dates = unique_dates_pslp[idx_pslp-lookback:idx_pslp]
    lookback_dates = [pd.to_datetime(d).strftime('%Y-%m-%d') for d in lookback_dates]
    print(f"Dates to consider for calculating PSLP: {lookback_dates}")
    for header in original_headers:
        print(f"Looking at {header}...")
        df[header + " PSLP"].at[date_str] = pd.concat([df[header].at[d].reset_index(drop=True) for d in lookback_dates], axis=1).mean(axis=1)
    return df

In [29]:
df_test_1 = _calculate_pslp(df_test, original_headers, "20180303")
df_test_2 = _calculate_pslp_2(df_test, original_headers, "20180303")

print(df_test_1["Actual Load PSLP"].loc["20180303"] - df_test_2["Actual Load PSLP"].loc["20180303"])
#print(df_test["Actual Load"].loc["20180303"])

Calculating PSLP for date 20180303...
PSLP category of 2018-03-03 is 1.
Index in unique days of PSLP category is 9.
Dates to consider for calculating PSLP: ['2018-02-10', '2018-02-17', '2018-02-24']
Looking at Actual Load...
Looking at Biomass Actual Aggregated...
Looking at Fossil Brown coal/Lignite Actual Aggregated...
Looking at Fossil Coal-derived gas Actual Aggregated...
Looking at Fossil Gas Actual Aggregated...
Looking at Fossil Hard coal Actual Aggregated...
Looking at Fossil Oil Actual Aggregated...
Looking at Geothermal Actual Aggregated...
Looking at Hydro Pumped Storage Actual Aggregated...
Looking at Hydro Run-of-river and poundage Actual Aggregated...
Looking at Hydro Water Reservoir Actual Aggregated...
Looking at Nuclear Actual Aggregated...
Looking at Other Actual Aggregated...
Looking at Other renewable Actual Aggregated...
Looking at Solar Actual Aggregated...
Looking at Waste Actual Aggregated...
Looking at Wind Offshore Actual Aggregated...
Looking at Wind Onshore 

In [18]:
def calculate_pslps(df, original_headers, date_str=None, lookback=3, country_code='DE', DEBUG=False):
    """
    Calculate PSLPs for all dates in dataframe or for given date from given data.
    
    The data is categorized into weekdays, Saturdays, and Sundays/holidays.
    The `lookback` most recent days from the specified date's category are used to
    calculate the corresponding PSLP as the average.
    
    Params
    ------
    df : pandas.Dataframe
         data to calculate PSLP for, must have datetime index
    original_headers : list of str
                       categories to calculate PSLP for
    date_str : str
               date 'YYYYMMDD' to calculate PSLP for; if None, calculate PSLP for all dates
    lookback : int
               number of days to consider in each category for calculating PSLP
    country_code : str
                   considered country (for holidays)
    
    """
    if date_str is not None:
        print(f"Calculating PSLP for date {date_str} only...")
        df = _calculate_pslp(df, 
                            original_headers, 
                            date_str, 
                            lookback=lookback, 
                            country_code=country_code, 
                            DEBUG=DEBUG)
    
    else:
        print("Calculating PSLPs for all dates in dataframe...")
        unique_dates = df.index.to_series().dt.date.drop_duplicates().tolist() # Get unique dates in data index.
        unique_dates = [pd.to_datetime(d).strftime('%Y-%m-%d') for d in unique_dates]
        
        for date in unique_dates:
            try:
                df = _calculate_pslp(df, 
                                    original_headers, 
                                    date_str=date, 
                                    lookback=lookback, 
                                    country_code=country_code, 
                                    DEBUG=DEBUG)
            except IndexError as e:
                print(e)
    return df

In [19]:
#date_str = "20181208"
df_Pslps= calculate_pslps(df_test, original_headers)#, date_str)
#print(df_Pslps["Actual Load PSLP"].loc[date_str])

Calculating PSLPs for all dates in dataframe...
Calculating PSLP for date 2017-12-25...
PSLP category of 2017-12-25 is 2.
Index in unique days of PSLP category is 0.
PSLP cannot be calculated. Less than 3 samples in PSLP category for date 2017-12-25.
Calculating PSLP for date 2017-12-26...
PSLP category of 2017-12-26 is 2.
Index in unique days of PSLP category is 1.
PSLP cannot be calculated. Less than 3 samples in PSLP category for date 2017-12-26.
Calculating PSLP for date 2017-12-27...
PSLP category of 2017-12-27 is 0.
Index in unique days of PSLP category is 0.
PSLP cannot be calculated. Less than 3 samples in PSLP category for date 2017-12-27.
Calculating PSLP for date 2017-12-28...
PSLP category of 2017-12-28 is 0.
Index in unique days of PSLP category is 1.
PSLP cannot be calculated. Less than 3 samples in PSLP category for date 2017-12-28.
Calculating PSLP for date 2017-12-29...
PSLP category of 2017-12-29 is 0.
Index in unique days of PSLP category is 2.
PSLP cannot be calcula

Calculating PSLP for date 2018-01-09...
PSLP category of 2018-01-09 is 0.
Index in unique days of PSLP category is 8.
Dates to consider for calculating PSLP: ['2018-01-04', '2018-01-05', '2018-01-08']
Looking at Actual Load...
Looking at Biomass Actual Aggregated...
Looking at Fossil Brown coal/Lignite Actual Aggregated...
Looking at Fossil Coal-derived gas Actual Aggregated...
Looking at Fossil Gas Actual Aggregated...
Looking at Fossil Hard coal Actual Aggregated...
Looking at Fossil Oil Actual Aggregated...
Looking at Geothermal Actual Aggregated...
Looking at Hydro Pumped Storage Actual Aggregated...
Looking at Hydro Run-of-river and poundage Actual Aggregated...
Looking at Hydro Water Reservoir Actual Aggregated...
Looking at Nuclear Actual Aggregated...
Looking at Other Actual Aggregated...
Looking at Other renewable Actual Aggregated...
Looking at Solar Actual Aggregated...
Looking at Waste Actual Aggregated...
Looking at Wind Offshore Actual Aggregated...
Looking at Wind Onshor

Calculating PSLP for date 2018-01-18...
PSLP category of 2018-01-18 is 0.
Index in unique days of PSLP category is 15.
Dates to consider for calculating PSLP: ['2018-01-15', '2018-01-16', '2018-01-17']
Looking at Actual Load...
Looking at Biomass Actual Aggregated...
Looking at Fossil Brown coal/Lignite Actual Aggregated...
Looking at Fossil Coal-derived gas Actual Aggregated...
Looking at Fossil Gas Actual Aggregated...
Looking at Fossil Hard coal Actual Aggregated...
Looking at Fossil Oil Actual Aggregated...
Looking at Geothermal Actual Aggregated...
Looking at Hydro Pumped Storage Actual Aggregated...
Looking at Hydro Run-of-river and poundage Actual Aggregated...
Looking at Hydro Water Reservoir Actual Aggregated...
Looking at Nuclear Actual Aggregated...
Looking at Other Actual Aggregated...
Looking at Other renewable Actual Aggregated...
Looking at Solar Actual Aggregated...
Looking at Waste Actual Aggregated...
Looking at Wind Offshore Actual Aggregated...
Looking at Wind Onsho

Calculating PSLP for date 2018-01-27...
PSLP category of 2018-01-27 is 1.
Index in unique days of PSLP category is 4.
Dates to consider for calculating PSLP: ['2018-01-06', '2018-01-13', '2018-01-20']
Looking at Actual Load...
Looking at Biomass Actual Aggregated...
Looking at Fossil Brown coal/Lignite Actual Aggregated...
Looking at Fossil Coal-derived gas Actual Aggregated...
Looking at Fossil Gas Actual Aggregated...
Looking at Fossil Hard coal Actual Aggregated...
Looking at Fossil Oil Actual Aggregated...
Looking at Geothermal Actual Aggregated...
Looking at Hydro Pumped Storage Actual Aggregated...
Looking at Hydro Run-of-river and poundage Actual Aggregated...
Looking at Hydro Water Reservoir Actual Aggregated...
Looking at Nuclear Actual Aggregated...
Looking at Other Actual Aggregated...
Looking at Other renewable Actual Aggregated...
Looking at Solar Actual Aggregated...
Looking at Waste Actual Aggregated...
Looking at Wind Offshore Actual Aggregated...
Looking at Wind Onshor

Calculating PSLP for date 2018-02-05...
PSLP category of 2018-02-05 is 0.
Index in unique days of PSLP category is 27.
Dates to consider for calculating PSLP: ['2018-01-31', '2018-02-01', '2018-02-02']
Looking at Actual Load...
Looking at Biomass Actual Aggregated...
Looking at Fossil Brown coal/Lignite Actual Aggregated...
Looking at Fossil Coal-derived gas Actual Aggregated...
Looking at Fossil Gas Actual Aggregated...
Looking at Fossil Hard coal Actual Aggregated...
Looking at Fossil Oil Actual Aggregated...
Looking at Geothermal Actual Aggregated...
Looking at Hydro Pumped Storage Actual Aggregated...
Looking at Hydro Run-of-river and poundage Actual Aggregated...
Looking at Hydro Water Reservoir Actual Aggregated...
Looking at Nuclear Actual Aggregated...
Looking at Other Actual Aggregated...
Looking at Other renewable Actual Aggregated...
Looking at Solar Actual Aggregated...
Looking at Waste Actual Aggregated...
Looking at Wind Offshore Actual Aggregated...
Looking at Wind Onsho

...

Calculating PSLP for date 2018-11-15...
PSLP category of 2018-11-15 is 0.
Index in unique days of PSLP category is 224.
Dates to consider for calculating PSLP: ['2018-11-12', '2018-11-13', '2018-11-14']
Looking at Actual Load...
Looking at Biomass Actual Aggregated...
Looking at Fossil Brown coal/Lignite Actual Aggregated...
Looking at Fossil Coal-derived gas Actual Aggregated...
Looking at Fossil Gas Actual Aggregated...
Looking at Fossil Hard coal Actual Aggregated...
Looking at Fossil Oil Actual Aggregated...
Looking at Geothermal Actual Aggregated...
Looking at Hydro Pumped Storage Actual Aggregated...
Looking at Hydro Run-of-river and poundage Actual Aggregated...
Looking at Hydro Water Reservoir Actual Aggregated...
Looking at Nuclear Actual Aggregated...
Looking at Other Actual Aggregated...
Looking at Other renewable Actual Aggregated...
Looking at Solar Actual Aggregated...
Looking at Waste Actual Aggregated...
Looking at Wind Offshore Actual Aggregated...
Looking at Wind Onsh

Calculating PSLP for date 2018-11-23...
PSLP category of 2018-11-23 is 0.
Index in unique days of PSLP category is 230.
Dates to consider for calculating PSLP: ['2018-11-20', '2018-11-21', '2018-11-22']
Looking at Actual Load...
Looking at Biomass Actual Aggregated...
Looking at Fossil Brown coal/Lignite Actual Aggregated...
Looking at Fossil Coal-derived gas Actual Aggregated...
Looking at Fossil Gas Actual Aggregated...
Looking at Fossil Hard coal Actual Aggregated...
Looking at Fossil Oil Actual Aggregated...
Looking at Geothermal Actual Aggregated...
Looking at Hydro Pumped Storage Actual Aggregated...
Looking at Hydro Run-of-river and poundage Actual Aggregated...
Looking at Hydro Water Reservoir Actual Aggregated...
Looking at Nuclear Actual Aggregated...
Looking at Other Actual Aggregated...
Looking at Other renewable Actual Aggregated...
Looking at Solar Actual Aggregated...
Looking at Waste Actual Aggregated...
Looking at Wind Offshore Actual Aggregated...
Looking at Wind Onsh

Calculating PSLP for date 2018-12-01...
PSLP category of 2018-12-01 is 1.
Index in unique days of PSLP category is 48.
Dates to consider for calculating PSLP: ['2018-11-10', '2018-11-17', '2018-11-24']
Looking at Actual Load...
Looking at Biomass Actual Aggregated...
Looking at Fossil Brown coal/Lignite Actual Aggregated...
Looking at Fossil Coal-derived gas Actual Aggregated...
Looking at Fossil Gas Actual Aggregated...
Looking at Fossil Hard coal Actual Aggregated...
Looking at Fossil Oil Actual Aggregated...
Looking at Geothermal Actual Aggregated...
Looking at Hydro Pumped Storage Actual Aggregated...
Looking at Hydro Run-of-river and poundage Actual Aggregated...
Looking at Hydro Water Reservoir Actual Aggregated...
Looking at Nuclear Actual Aggregated...
Looking at Other Actual Aggregated...
Looking at Other renewable Actual Aggregated...
Looking at Solar Actual Aggregated...
Looking at Waste Actual Aggregated...
Looking at Wind Offshore Actual Aggregated...
Looking at Wind Onsho

Calculating PSLP for date 2018-12-09...
PSLP category of 2018-12-09 is 2.
Index in unique days of PSLP category is 58.
Dates to consider for calculating PSLP: ['2018-11-18', '2018-11-25', '2018-12-02']
Looking at Actual Load...
Looking at Biomass Actual Aggregated...
Looking at Fossil Brown coal/Lignite Actual Aggregated...
Looking at Fossil Coal-derived gas Actual Aggregated...
Looking at Fossil Gas Actual Aggregated...
Looking at Fossil Hard coal Actual Aggregated...
Looking at Fossil Oil Actual Aggregated...
Looking at Geothermal Actual Aggregated...
Looking at Hydro Pumped Storage Actual Aggregated...
Looking at Hydro Run-of-river and poundage Actual Aggregated...
Looking at Hydro Water Reservoir Actual Aggregated...
Looking at Nuclear Actual Aggregated...
Looking at Other Actual Aggregated...
Looking at Other renewable Actual Aggregated...
Looking at Solar Actual Aggregated...
Looking at Waste Actual Aggregated...
Looking at Wind Offshore Actual Aggregated...
Looking at Wind Onsho

Calculating PSLP for date 2018-12-17...
PSLP category of 2018-12-17 is 0.
Index in unique days of PSLP category is 246.
Dates to consider for calculating PSLP: ['2018-12-12', '2018-12-13', '2018-12-14']
Looking at Actual Load...
Looking at Biomass Actual Aggregated...
Looking at Fossil Brown coal/Lignite Actual Aggregated...
Looking at Fossil Coal-derived gas Actual Aggregated...
Looking at Fossil Gas Actual Aggregated...
Looking at Fossil Hard coal Actual Aggregated...
Looking at Fossil Oil Actual Aggregated...
Looking at Geothermal Actual Aggregated...
Looking at Hydro Pumped Storage Actual Aggregated...
Looking at Hydro Run-of-river and poundage Actual Aggregated...
Looking at Hydro Water Reservoir Actual Aggregated...
Looking at Nuclear Actual Aggregated...
Looking at Other Actual Aggregated...
Looking at Other renewable Actual Aggregated...
Looking at Solar Actual Aggregated...
Looking at Waste Actual Aggregated...
Looking at Wind Offshore Actual Aggregated...
Looking at Wind Onsh

In [22]:
print(df_Pslps.loc["20181028"])

                           Actual Load  Biomass Actual Aggregated  \
2018-10-28 00:00:00+01:00      43205.0                     4720.0   
2018-10-28 00:15:00+01:00      42955.0                     4712.0   
2018-10-28 00:30:00+01:00      42393.0                     4719.0   
2018-10-28 00:45:00+01:00      42369.0                     4723.0   
2018-10-28 01:00:00+01:00      41684.0                     4722.0   
2018-10-28 01:15:00+01:00      41476.0                     4727.0   
2018-10-28 01:30:00+01:00      41310.0                     4725.0   
2018-10-28 01:45:00+01:00      41179.0                     4716.0   
2018-10-28 02:00:00+01:00      41046.0                     4728.0   
2018-10-28 02:15:00+01:00      40891.0                     4725.0   
2018-10-28 02:30:00+01:00      41013.0                     4726.0   
2018-10-28 02:45:00+01:00      40933.0                     4717.0   
2018-10-28 03:00:00+01:00      41167.0                     4725.0   
2018-10-28 03:15:00+01:00      412

In [ ]:
#print(df_Pslps)
df_Pslps["Actual Load Residuals"] = df_Pslps["Actual Load"] - df_Pslps["Actual Load PSLP"]
print(df_Pslps["Actual Load Residuals"].loc[date_str])
print(df_Pslps["Actual Load"].loc[date_str])
print(df_Pslps["Actual Load PSLP"].loc[date_str])

In [ ]:
## TODO Plot PSLPs.

### Complete parameter list
https://transparency.entsoe.eu/content/static_content/Static%20content/web%20api/Guide.html#_complete_parameter_list

### Queries returning Pandas Series

`client.query_day_ahead_prices(country_code, start=start,end=end)` <br>
`client.query_net_position(country_code, start=start, end=end, dayahead=True)` <br>
`client.query_crossborder_flows(country_code_from, country_code_to, start, end)` <br>
`client.query_scheduled_exchanges(country_code_from, country_code_to, start, end, dayahead=False)` <br>
`client.query_net_transfer_capacity_dayahead(country_code_from, country_code_to, start, end)` <br>
`client.query_net_transfer_capacity_weekahead(country_code_from, country_code_to, start, end)` <br>
`client.query_net_transfer_capacity_monthahead(country_code_from, country_code_to, start, end)` <br>
`client.query_net_transfer_capacity_yearahead(country_code_from, country_code_to, start, end)` <br>
`client.query_intraday_offered_capacity(country_code_from, country_code_to, start, end,implicit=True)` <br>
`client.query_offered_capacity(country_code_from, country_code_to, start, end, contract_marketagreement_type, implicit=True)` <br>
`client.query_aggregate_water_reservoirs_and_hydro_storage(country_code, start, end)`

### Queries returning Pandas DataFrames

`client.query_load(country_code, start=start,end=end)` <br>
`client.query_load_forecast(country_code, start=start,end=end)` <br>
`client.query_load_and_forecast(country_code, start=start, end=end)` <br>
`client.query_generation_forecast(country_code, start=start,end=end)` <br>
`client.query_wind_and_solar_forecast(country_code, start=start,end=end, psr_type=None)` <br>
`client.query_generation(country_code, start=start,end=end, psr_type=None)` <br>
`client.query_generation_per_plant(country_code, start=start,end=end, psr_type=None)` <br>
`client.query_installed_generation_capacity(country_code, start=start,end=end, psr_type=None)` <br>
`client.query_installed_generation_capacity_per_unit(country_code, start=start,end=end, psr_type=None)` <br>
`client.query_imbalance_prices(country_code, start=start,end=end, psr_type=None)` <br>
`client.query_contracted_reserve_prices(country_code, start, end, type_marketagreement_type, psr_type=None)` <br>
`client.query_contracted_reserve_amount(country_code, start, end, type_marketagreement_type, psr_type=None)` <br>
`client.query_unavailability_of_generation_units(country_code, start=start,end=end, docstatus=None, periodstartupdate=None, periodendupdate=None)` <br>
`client.query_unavailability_of_production_units(country_code, start, end, docstatus=None, periodstartupdate=None, periodendupdate=None)` <br>
`client.query_unavailability_transmission(country_code_from, country_code_to, start, end, docstatus=None, periodstartupdate=None, periodendupdate=None)` <br>
`client.query_withdrawn_unavailability_of_generation_units(country_code, start, end)` <br>
`client.query_import(country_code, start, end)` <br>
`client.query_generation_import(country_code, start, end)` <br>
`client.query_procured_balancing_capacity(country_code, start, end, process_type, type_marketagreement_type=None)`

## Load data from client

In [ ]:
df = pd.DataFrame()
df["load forecast"] = client.query_load_forecast(country_code, start=start,end=end)
df["load"] = client.query_load(country_code, start=start,end=end)
df["load forecast error"] = df["load forecast"] - df["load"]
df["generation forecast"] = client.query_generation_forecast(country_code, start=start,end=end)

In [ ]:
df_gen = client.query_generation(country_code, start=start,end=end, psr_type=None)
df_gen.head()

In [ ]:
df_gen.xs(key="Actual Aggregated", level=1, axis=1)

In [ ]:
fig, ax = plt.subplots(figsize=(14,7))
ax.plot(df_gen.xs(key="Actual Aggregated", level=1, axis=1))
ax.legend()

In [ ]:
df["generation"] = df_gen.sum(axis=1)

In [ ]:
df["generation forecast error"] = df["generation forecast"] - df["generation"]

In [ ]:
px.line(df)

## Save to csv file

In [ ]:
df.to_csv('entsoe.csv')